# Demo for face verification app

# 1. Dependencies and CAM ID setup

We have completed training some model and store those models to `model_saved` folder. Now we load those models and use them to verify the face of a person.

The process will be as follow:

1. User register their face to the system through sacnning process. After we get sanning images, extract face then extract face embeddings and store them to database.

2. When user want to verify their face, we open the camera, capture the image, extract face embeddings and compare with the embeddings in database (with correspoding name use provide when login).

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import random
import uuid
from PIL import Image
from mtcnn.mtcnn import MTCNN
from numpy import savez_compressed
import pickle
import json
import os

# For the Facenet model
import torch  # Ensure torch is imported here to avoid circular import issues
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms

Each device has a different camera IDs/index, so we need to find the correct camera ID for our device. We try to loop throught a range of camera IDs and ask user to check if the camera is working. Each devices can have many webcams, so we have many corresponding camera IDs, we ask user to choose their prefered camera ID as well.

Store these congifuration to the `application_data/setting.json` file. **Next time, when user open the app, these setting will be loaded without asking user again.**

In [14]:
# Create a application_data folder to store all app related data
os.makedirs('application_data', exist_ok=True)

# Setting.jpg file path
SETTINGS_FILE_PATH = os.path.join('application_data', 'settings.json')


In [24]:
def detect_cameras():
   
    print("Scanning for available cameras")
    detected_cameras = []

    # Test cameras 0-9
    for cam_id in range(10):
        print(f"\nTesting camera {cam_id}")
        cap = cv2.VideoCapture(cam_id)


    
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            cv2.imshow('Detection camera, if see the camera, press q to quite then type Y to continue', frame)
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

        # Check if camera is detected
        response = input(f"Can you see camera ID {cam_id}? (Y/n): ").lower()
        if response == 'y' :
            detected_cameras.append(cam_id)


    if not detected_cameras:
        print("No cameras detected!")
        return [], None

    # Select preferred camera
    preferred = None
    if len(detected_cameras) > 1:
        while preferred not in detected_cameras:
            try:
                print("\nAll available cameras:", detected_cameras)
                preferred = int(input("Enter the camera ID you want to use (ID 0 is often RGB camera, ID 2 is often IR camera): "))
            except ValueError:
                print("Please enter a valid number")
    else:
        preferred = detected_cameras[0]

    # Save settings
    settings = {
        "camera_list": detected_cameras,
        "preferred_camera": preferred
    }
    
    with open(SETTINGS_FILE_PATH, 'w') as f:
        json.dump(settings, f)
        print(f"\nSettings saved to {SETTINGS_FILE_PATH}")

    return detected_cameras, preferred

When detecting cameras, OpenCV can only open the RGB camera (index 0) and cannot open the IR camera (often index 2) on Windows. On Linux, it can detect all cameras normally.


**Next time, just use Linux to develop these things :))**

In [25]:
CAM_ID = 0 # Default camera ID

# Check if settings file exists, if no then call the detect camera function
if not os.path.exists(SETTINGS_FILE_PATH):
    cameras, preferred = detect_cameras()
    print(f"\nDetected cameras: {cameras}")
    print(f"Preferred camera: {preferred}")
else:
    print(f"Settings file found at {SETTINGS_FILE_PATH}, if you want to rescan for cameras delete this file.")


Scanning for available cameras

Testing camera 0

Testing camera 1

Testing camera 2

Testing camera 3

Testing camera 4

Testing camera 5

Testing camera 6

Testing camera 7

Testing camera 8

Testing camera 9

Settings saved to application_data\settings.json

Detected cameras: [0]
Preferred camera: 0


In [18]:
#Load the settings file
with open(SETTINGS_FILE_PATH, 'r') as f:
    settings = json.load(f)
    CAM_ID = settings['preferred_camera']
    
print("Running app with camera ID:", CAM_ID)

Runnin app with camera ID: 0


Here are some example of camera IDs:

- CAM_ID = 0 for laptop normal webcam
- CAM_ID = 2 for laptop IR webcam
- CAM_ID = 4 for external webcam


***Depend on each devices, these number can be different. Try out all number start from 0 and see which one is the correct one on your device.***


# 2. Enrollment process

In [19]:
# Create a application_data folder to store all app related data

os.makedirs('application_data', exist_ok=True)

# Inside this foilder, create a folder name validation_images to store all the images that are used for validation process
valiation_images = os.path.join('application_data', 'validation_images')
os.makedirs(valiation_images, exist_ok=True)

In [20]:
# Connect to the camera and take pictures of the user for sacnning process
# Save the images in the validation_images folder, inside a subfolder with the user's name

# Function to capture images from webcam
def capture_images(user_name, store_location='application_data/validation_images'):
    user_folder = os.path.join(store_location, user_name)
    os.makedirs(user_folder, exist_ok=True)
    
    cap = cv2.VideoCapture(CAM_ID)
    print("Press 'p' to capture an image, 'q' to quit.")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        cv2.imshow('Enrollment process, p to capture, q to quit', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('p'):
            img_name = f"{uuid.uuid4()}.jpg"
            img_path = os.path.join(user_folder, img_name)
            cv2.imwrite(img_path, frame)
            print(f"Image saved: {img_path}")
        elif key == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Ask for user name and capture images
user_name = input("Enter your name to register to system: ")
capture_images(user_name)

Press 'p' to capture an image, 'q' to quit.
Image saved: application_data/validation_images\cta\e4377245-23da-4d8f-a601-6cc65356e2ea.jpg
Image saved: application_data/validation_images\cta\0121aa70-0d6c-4d07-863a-3eb9851ec341.jpg
Image saved: application_data/validation_images\cta\d540c812-3415-4605-bdab-e3c0506dec10.jpg
Image saved: application_data/validation_images\cta\b7d26d56-7873-4e2f-9f33-40718ff5b281.jpg
Image saved: application_data/validation_images\cta\14b8a8c8-7c41-4812-9402-f317111357d6.jpg
Image saved: application_data/validation_images\cta\cb836c96-1ea9-4dc6-bb4d-3958f7d42b16.jpg
Image saved: application_data/validation_images\cta\48a14e35-fc12-4b49-accc-212e4b48cc4b.jpg
Image saved: application_data/validation_images\cta\aa576e20-9989-44be-9914-948533db7892.jpg
Image saved: application_data/validation_images\cta\9d36019e-2284-425f-87b2-b1fe2028e501.jpg
Image saved: application_data/validation_images\cta\edc56997-24e8-4260-8dba-26a2f5b1b77f.jpg


In [21]:
# Now, we process the images to extract the face of a subfolder/person name in the validation_images folder
# then store the faces.npz right in that subfolder, using MTCNN to detect faces

# Function to detect faces and save to faces.npz
# Parameters:
# user_name: Name of the user whose images are to be processed
def detect_and_save_faces(user_name, store_location='application_data/validation_images'):
    user_folder = os.path.join(store_location, user_name)
    face_folder = os.path.join(user_folder, 'face') # A subfolder with person name already contains
    # sacnning images, so make a seperate `face` subfolder isnide that to sotre the faces.npz for better organization
    os.makedirs(face_folder, exist_ok=True)
    
    detector = MTCNN()
    faces = []
    
    for img_file in os.listdir(user_folder): # Loop through all the images in the user folder
        if img_file.endswith('.jpg'):
            img_path = os.path.join(user_folder, img_file)
            image = Image.open(img_path).convert('RGB')
            image_np = np.array(image)
            detections = detector.detect_faces(image_np)
            
            for i, detection in enumerate(detections):
                x, y, width, height = detection['box']
                face = image_np[y:y+height, x:x+width]
                face_image = Image.fromarray(face).resize((160, 160))
                face_array = np.array(face_image)
                faces.append(face_array)
    
    faces = np.array(faces)
    savez_compressed(os.path.join(face_folder, 'faces.npz'), faces)


# Call the function to detect faces and save to faces.npz
if user_name:  #username is input from the user at the previous step
    detect_and_save_faces(user_name)



The structure is as follow:

```plaintext
application_data
|
|───Validation_images
|   |───user1
|   |   |───face
|   |   |   └───faces.npz
|   |   |───image1.jpg
|   |   |───image2.jpg
|   |   |───...
```

In [22]:
# After the above steps, we have the faces.npz file for the user, from that file, we continue
# to extract the face embeddings

# Load the pre-trained FaceNet model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
facenet_model = InceptionResnetV1(pretrained='vggface2').eval().to(device)


# Define a function to generate embeddings (this function is already defined in
# the Preprocessing Notebook, so we can just copy it here)
# Parameters:
# - image_array: a numpy array representing the image
def generate_embedding(image_array, model=facenet_model):
    # Convert numpy array to PIL image
    image = Image.fromarray(image_array)
    # Preprocess the image
    transform = transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img_tensor = transform(image).unsqueeze(0).to(device)
    
    # Generate embedding
    with torch.no_grad():
        embedding = model(img_tensor).cpu().numpy()
    return embedding

# Function to generate embeddings and save to embedding.npz
def generate_and_save_embeddings(user_name, store_location='application_data/validation_images'):
    user_folder = os.path.join(store_location, user_name)
    face_folder = os.path.join(user_folder, 'face')
    embedding_folder = os.path.join(user_folder, 'embeddings')
    if not os.path.exists(embedding_folder):
        os.makedirs(embedding_folder)
    
    data = np.load(os.path.join(face_folder, 'faces.npz'))
    faces = data['arr_0']
    embeddings = []
    
    for face in faces:
        embedding = generate_embedding(face)
        embeddings.append(embedding)
    
    embeddings = np.array(embeddings)
    savez_compressed(os.path.join(embedding_folder, 'embeddings.npz'), embeddings)

# Call the function to generate embeddings and save to embeddings.npz
if user_name: # if user_name not null
    generate_and_save_embeddings(user_name)


The structure is as follow:

```plaintext
application_data
|
|───Validation_images
|   |───user1
|   |   |───face
|   |   |   └───faces.npz
|   |   |
|   |   |───embeddings
|   |   |   └───embeddings.npz
|   |   |
|   |   |───image1.jpg
|   |   |───image2.jpg
|   |   |───...
```

The final goal is just the `embeddings.npz` file, other face.npz, images are just for vizuale the pipeline.

# 3. Verification process

In [23]:
# When use wnat to login, capture a image from the webcam when user press 'v' and 
# then compare that input image with the all the embeddings of the user to check if the user is the same person

# Function to capture a single image directly from webcam, return the frame object
def verify_user(username, detector, facenet_model):


    # Frist, check if user existed in the system
    if not os.path.exists(os.path.join('application_data/validation_images', username)):
        print("User not found")
        # Then return an empty list
        return []

    cap = cv2.VideoCapture(CAM_ID)
    # Declare a variable frame to store the captured image
    frame = None

    
    result = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        cv2.imshow('Verify user. Press v to capture an image', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('v'):
            embedding_of_input = extract_face_and_generate_embedding(frame, detector, facenet_model)
            result = compare_embeddings(username,embedding_of_input, svm_model, scaler)
            break
    cap.release()
    cv2.destroyAllWindows()

    return result

   

# After capture image, extract the face from the image and generate the embeddings
def extract_face_and_generate_embedding(frame, detector, facenet_model):
    
    faces = []
    
    image = Image.fromarray(frame).convert('RGB')
    image = np.array(image)
    detections = detector.detect_faces(image)
    
    for i, detection in enumerate(detections):
        x, y, width, height = detection['box']
        face = image[y:y+height, x:x+width]
        face_image = Image.fromarray(face).resize((160, 160))
        face_array = np.array(face_image)
        faces.append(face_array)
    
    faces = np.array(faces)
    embedding = generate_embedding(faces[0], facenet_model) # Since we are capturing a single image, we only have one face
    return embedding

def generate_embedding(image_array, model=facenet_model):
    # Convert numpy array to PIL image
    image = Image.fromarray(image_array)
    # Preprocess the image
    transform = transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img_tensor = transform(image).unsqueeze(0).to(device)
    
    # Generate embedding
    with torch.no_grad():
        embedding = model(img_tensor).cpu().numpy()
    return embedding


# Function to compare the input image with all embeddings of the user using the model we pass in

def compare_embeddings(username, embedding_of_input, svm_model, scaler):
    user_folder = os.path.join('application_data/validation_images', username)
    embedding_folder = os.path.join(user_folder, 'embeddings')
    

    data = np.load(os.path.join(embedding_folder, 'embeddings.npz'))
    embeddings = data['arr_0']
    
    results = []
    
    for validation_embedding in embeddings:
        
        # Flatten the embeddings
        input_embedding_flat = embedding_of_input.flatten()
        validation_embedding_flat = validation_embedding.flatten()
        
        pair = np.concatenate((input_embedding_flat, validation_embedding_flat))
        pair_scaled = scaler.transform([pair])
        
        prediction = svm_model.predict(pair_scaled)
        probabilities = svm_model.predict_proba(pair_scaled)
        print("Compare with validation image, rresult is:", prediction[0], 'with confidence: ', probabilities)
        results.append(prediction[0])
    
    return results


# Main function

# Declare all model to use
detector = MTCNN()
svm_model = None
scaler = None
# Load the pre-trained FaceNet model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
facenet_model = InceptionResnetV1(pretrained='vggface2').eval().to(device)



with open('./model_saved/scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)

with open('./model_saved/svm_model.pkl', 'rb') as f:
        svm_model = pickle.load(f)

# Prompt who are trying to login
username = input("Who are you ?")
result = []
if username: # prevent the case user enter nothing
    result = verify_user(username, detector, facenet_model)
    print(result)


# Now, based on the result, we can decide if the user is the same person or not with a threshold. 
# If the proportion of output 1 / total output is greater than a threshold, we can say the user is the same person
# Define a threshold
threshold = 0.8

# Calculate the proportion of positive identifications
positive_identifications = sum(result)
total_identifications = len(result)

if total_identifications == 0:
    print("No face detected in the input image or the user is not found in the system")
else: 
    proportion = positive_identifications / total_identifications

    # Determine if the user is the same person
    if proportion > threshold:
        print("User verified successfully.")
    else:
        print("User verification failed.")


Compare with validation image, rresult is: 1 with confidence:  [[0.00422016 0.99577984]]
Compare with validation image, rresult is: 1 with confidence:  [[0.00609581 0.99390419]]
Compare with validation image, rresult is: 1 with confidence:  [[0.01743164 0.98256836]]
Compare with validation image, rresult is: 1 with confidence:  [[0.00371654 0.99628346]]
Compare with validation image, rresult is: 1 with confidence:  [[0.03855821 0.96144179]]
Compare with validation image, rresult is: 1 with confidence:  [[0.04620132 0.95379868]]
Compare with validation image, rresult is: 1 with confidence:  [[0.02192664 0.97807336]]
Compare with validation image, rresult is: 1 with confidence:  [[0.00865865 0.99134135]]
Compare with validation image, rresult is: 1 with confidence:  [[0.04487673 0.95512327]]
Compare with validation image, rresult is: 1 with confidence:  [[0.00908843 0.99091157]]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
User verified successfully.


In [ ]:
print(len(result))